$$ ITI \space AI-Pro: \space Intake \space 45 $$
$$ Recommender \space Systems $$
$$ Lab \space no. \space 3 $$

# `01` Import Necessary Libraries

In [4]:
# %pip install --no-cache-dir --force-reinstall numpy==1.23.5 scipy==1.9.3
# %pip install scikit-surprise==1.1.3

## `i` Default Libraries

In [5]:
import numpy as np
import pandas as pd
from surprise.reader import Reader
from surprise.dataset import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise.prediction_algorithms.matrix_factorization import SVD

----------------------------

# `02` Load Data

Load `songsDataset.csv` file into a dataframe

In [6]:
data = pd.read_csv("songsDataset.csv")
data.head()

,userID,songID,rating
0,0,90409,5
1,4,91266,1
2,5,8063,2
3,5,24427,4
4,5,105433,4


--------------------------

# `03` Matrix Factorization using Gradient Descent

Practice for Matrix Factorization Implementation

**Matrix Factorization Mathematical Derivation**

We know that the matrix factorization breaks the rating matrix $R$ into two matrices $U \in \mathbb{R}^{\#users \times K}$ and $M \in \mathbb{R}^{K \times \#items}$ where K represent the latent space dimensionality.

$R$ can then be approximated through the following equation:

$$
\mathbf{R} \approx \mathbf{U} \times \mathbf{M} = \hat{\mathbf{R}}
$$

The error, incorporating the regularization term, is calculated as follows:

$$
e_{ij}^2 = (r_{ij} - \sum_{k=1}^K{u_{ik}m_{kj}})^2 + \frac{\beta}{2} \sum_{k=1}^K{(||U||^2 + ||M||^2)}
$$

In order to be able to use Stochastic Gradient Descent (SGD) to optimize $U$ and $M$, we need to find the partial derivatives of the error function with respect to both $u_{ik}$ and $m_{kj}$. The partial derivatives can be derived as follows:

$$
\frac{\partial}{\partial u_{ik}}e_{ij}^2 = -2(r_{ij} - \hat{r}_{ij})(m_{kj}) + \frac{\beta}{2} \times (2 u_{ik}) = -2 e_{ij} m_{kj} + \beta u_{ik}
$$

$$
\frac{\partial}{\partial m_{ik}}e_{ij}^2 = -2(r_{ij} - \hat{r}_{ij})(u_{ik}) + \frac{\beta}{2} \times (2 m_{kj}) = -2 e_{ij} u_{ik} + \beta m_{kj}
$$

Thus the update rules will be:

$$
u'_{ik} = u_{ik} + \alpha \frac{\partial}{\partial u_{ik}}e_{ij}^2 = u_{ik} - \alpha(-2 e_{ij} m_{kj} + \beta u_{ik} ) = u_{ik} + \alpha(2 e_{ij} m_{kj} - \beta u_{ik} )
$$

$$
m'_{kj} = m_{kj} + \alpha \frac{\partial}{\partial m_{kj}}e_{ij}^2 = m_{kj} - \alpha(-2 e_{ij} u_{ik} + \beta m_{kj} ) = m_{kj} + \alpha(2 e_{ij} u_{ik} - \beta m_{kj} )
$$

## `0` Construct Utility Matrix from the Data

In [8]:
utility_matrix = data.pivot_table(index='userID', columns='songID', values='rating', fill_value=0)
print(utility_matrix.shape)
utility_matrix

(53963, 56)


songID,2263,2726,3785,8063,12709,13859,16548,17029,19299,19670,...,113954,119103,120147,122065,123176,125557,126757,131048,132189,134732
userID,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199976,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0
199980,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
199988,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## `i` Matrices Initialization

Initialize the two random weights matrices $U$ and $M$ (Try $K=3$)

**Note**: Refer to the next cell for the dimensions of $U$ and $M$

**Hine**: You may use a function from `numpy.random` module (see, [Documentation](https://numpy.org/doc/stable/reference/random/index.html))

In [9]:
num_users, num_items = utility_matrix.shape
K = 3

U = np.random.rand(num_users, K)
M = np.random.rand(K, num_items)

print("U shape:", U.shape)
print("M shape:", M.shape)

U shape: (53963, 3)
M shape: (3, 56)


## `ii` Define a Function to Implement Matrix Factorization

**Function Parameters**:
- `R`: Utility Matrix [of shape: (no. of users, no. of items)]
- `U`: User Latent Features Array [of shape: (no. of users, K)]
- `M`: Items Latent Features Array [of shape: (K, no. of items)]
- `epochs`: No. of training epochs
- `lr`: Learning rate (alpha)
- `beta`: Regularization Parameter

**Function Output**:
- `U`: Optimized User Latent Features Array
- `M`: Optimized Items Latent Features Array

**Main Procedures**:
1. Calculate predicted ratings
2. Calculate MSE Error
3. Calculate gradients
4. Update $U$ and $M$


In [10]:
def matrix_factorization(R: np.ndarray, U: np.ndarray, M: np.ndarray, epochs: int, lr: float, beta: float):
    """
    Parameters:
    - R: Utility matrix (users x items)
    - U: User latent feature matrix (users x K)
    - M: Item latent feature matrix (K x items)
    - epochs: Number of training iterations
    - lr: Learning rate (alpha)
    - beta: Regularization parameter

    Returns:
    - Optimized U and M
    """

    # Confirm that no. of features is consistent between U and M
    assert U.shape[1] == M.shape[0], f'U and M must have consistent K. Found K={U.shape[1]} for U and K={M.shape[0]} for M'

    # Extract No. of Features (K)
    num_users, num_items = R.shape
    K = U.shape[1]

    # Define the Epochs loop
    for epoch in range(epochs):
        # Loop over every element in R
        # Stochastic Gradient Descent
        for i in range(num_users):   # Loop over each user
            for j in range(num_items):  # Loop over each item
                if R[i, j] > 0:   # Only proceed if the current interaction (i, j) is not missing
                    # Prediction and error
                    prediction = np.dot(U[i, :], M[:, j])
                    eij = R[i, j] - prediction  # Calculate the error in prediction

                    # Update latent factors
                    for k in range(K):   # Loop over each latent features dimension
                        # Update Rules for both U and M:
                        U[i, k] += lr * (2 * eij * M[k, j] - beta * U[i, k])
                        M[k, j] += lr * (2 * eij * U[i, k] - beta * M[k, j])


        ## Error Calculation ##
        # Compute total error
        e_last = 1e10 if epoch == 0 else e
        e = 0    # Initialize a variable to accumelate the errors
        for i in range(num_users):   # Loop over each user
            for j in range(num_items):   # Loop over each item
                if R[i, j] > 0:   # Only proceed if the current interaction (i, j) is not missing
                    prediction = np.dot(U[i, :], M[:, j])   # since we only calculate the error for interactions having a ground truth value
                    first_part = (R[i, j] - prediction) ** 2   # calculate the first part of the error
                    second_part = 0   # Initialize a variable to accumelate the second part of the error
                    for k in range(K):   # Loop over each latent features dimension
                        second_part += (beta / 2) * (U[i, k] ** 2 + M[k, j] ** 2)
                    e += first_part + second_part   # accumelate the error to the total error

        print(f'Epoch {epoch+1}/{epochs}: Total Error = {e:.4f}')   # Stop if error is so small or improvement is not significant
        if e < 0.001 or abs(e_last - e) < 0.001:
            break

    return U, M


### `[Bonus]` Vectorized Error Calculation

Can the error calculation part be vectorized to get rid of for loops?

If you would like a challenge, try to redefine the function in the next cell with a vectorized error calculation.

## `iii` Use the Function to to Optimize the $U$ and $V$

In [11]:
U, M = matrix_factorization(R=utility_matrix.values, U=U, M=M, epochs=100, lr=0.01, beta=0.01)

Epoch 1/100: Total Error = 123348.8351
Epoch 2/100: Total Error = 71582.9471
Epoch 3/100: Total Error = 50412.9557
Epoch 4/100: Total Error = 40906.4890
Epoch 5/100: Total Error = 36029.9522
Epoch 6/100: Total Error = 32990.5904
Epoch 7/100: Total Error = 30668.7368
Epoch 8/100: Total Error = 28650.7204
Epoch 9/100: Total Error = 26818.3955
Epoch 10/100: Total Error = 25158.9118
Epoch 11/100: Total Error = 23680.5759
Epoch 12/100: Total Error = 22384.1164
Epoch 13/100: Total Error = 21258.8547
Epoch 14/100: Total Error = 20286.7390
Epoch 15/100: Total Error = 19446.9782
Epoch 16/100: Total Error = 18719.2124
Epoch 17/100: Total Error = 18085.1719
Epoch 18/100: Total Error = 17529.2877
Epoch 19/100: Total Error = 17038.6952
Epoch 20/100: Total Error = 16602.9444
Epoch 21/100: Total Error = 16213.6053
Epoch 22/100: Total Error = 15863.8709
Epoch 23/100: Total Error = 15548.2048
Epoch 24/100: Total Error = 15262.0493
Epoch 25/100: Total Error = 15001.5956
Epoch 26/100: Total Error = 14763

## `iv` Recommend top-K Songs

Recommend top-K ($K=5$) songs for user ($userID=199988$)

Note: Make sure to filter songs they already rated before

In [12]:
def recommend_top_k_songs(user_id, utility_matrix, U, M, K=5):
    # Get the index of the user in the utility matrix
    if user_id not in utility_matrix.index:
        print(f"User {user_id} not found in utility matrix.")
        return

    user_idx = utility_matrix.index.get_loc(user_id)
    user_ratings = utility_matrix.loc[user_id].values
    predicted_ratings = np.dot(U[user_idx, :], M)

    # Mask already-rated songs
    unrated_mask = user_ratings == 0
    predicted_ratings_unrated = predicted_ratings * unrated_mask

    # Get top-K recommendations
    top_k_indices = np.argsort(predicted_ratings_unrated)[::-1][:K]
    top_k_song_ids = utility_matrix.columns[top_k_indices]
    top_k_scores = predicted_ratings[top_k_indices]

    print(f"Top {K} Recommended Items for User {user_id}:")
    for i, (song_id, score) in enumerate(zip(top_k_song_ids, top_k_scores), 1):
        print(f" - Top {i} Song: {song_id} (Predicted Rating: {score})")

In [13]:
recommend_top_k_songs(user_id=199988, utility_matrix=utility_matrix, U=U, M=M, K=5)

Top 5 Recommended Items for User 199988:
 - Top 1 Song: 24427 (Predicted Rating: 5.669130492252588)
 - Top 2 Song: 62954 (Predicted Rating: 5.488730192600653)
 - Top 3 Song: 2263 (Predicted Rating: 5.488580694396972)
 - Top 4 Song: 52611 (Predicted Rating: 5.420585754223022)
 - Top 5 Song: 60888 (Predicted Rating: 5.270722407592803)


------------------------

# `04` Matrix Factorization using SVD Algorithm

Practice for using `SVD` algorithm implementation from `scikit surprise` library.

## `i` Prepare the Data

- Load the Data into `surprise` Dataset
- Split data into train and test


In [14]:
data['rating'].min(), data['rating'].max()

(1, 5)

In [16]:
reader = Reader(rating_scale=(1, 5))

surprise_data = Dataset.load_from_df(data[['userID', 'songID', 'rating']], reader)

trainset, testset = train_test_split(surprise_data, test_size=0.2, random_state=99)

trainset.n_users, trainset.n_items, trainset.n_ratings

(45602, 56, 57636)

## `ii` Model Initialization

Instantiate two models:
- Model with baselines (biases)
- Model without baselines

**Note**: Use `surprise.prediction_algorithms.matrix_factorization.SVD` (see, [Documentation](https://surprise.readthedocs.io/en/stable/matrix_factorization.html#:~:text=surprise.prediction_algorithms.matrix_factorization.-,SVD,-(n_factors%3D)))

In [17]:
# Biased Model
model_with_bias = SVD(biased=True)

In [18]:
# Non-Biased Model
model_without_bias = SVD(biased=False)

## `iii` Fit each Model on Training Data

In [19]:
# Biased Model
model_with_bias.fit(trainset)

In [20]:
# Non-Biased Model
model_without_bias.fit(trainset)

## `iv` Test both Models on the Testing Data

Compare the errors of the two models using multiple error formulas.

**Note**: Refer to `surprise.accuracy` module (see, [Documentation](https://surprise.readthedocs.io/en/stable/accuracy.html))

In [21]:
# Biased Model
predictions_with_bias = model_with_bias.test(testset)

In [22]:
# Evaluation for the model with bais
rmse_with = accuracy.rmse(predictions_with_bias)
mae_with = accuracy.mae(predictions_with_bias)
mse_with = accuracy.mse(predictions_with_bias)
fcp_with = accuracy.fcp(predictions_with_bias)

RMSE: 1.4957
MAE:  1.2948
MSE: 2.2373
FCP:  0.5042


In [23]:
# Non-Biased Model
predictions_without_bias = model_without_bias.test(testset)

In [24]:
# Evaluate model with no bias
rmse_without = accuracy.rmse(predictions_without_bias)
mae_without = accuracy.mae(predictions_without_bias)
mse_without = accuracy.mse(predictions_without_bias)
fcp_without = accuracy.fcp(predictions_without_bias)

RMSE: 2.1081
MAE:  1.7514
MSE: 4.4440
FCP:  0.4911


## `v` Recommend Top $10$ Songs for User $199988$

Is there a difference in recommended songs from the two models?

In [25]:
all_songs = data[['songID']].drop_duplicates()

In [26]:
user_history = data[data['userID'] == 199988][['songID']].drop_duplicates()

In [27]:
unseen_songs = all_songs[~all_songs['songID'].isin(user_history['songID'])]
unseen_songs.head()

,songID
0,90409
1,91266
2,8063
3,24427
4,105433


In [28]:
# Biased Model
unseen_songs.loc[:, 'predicted_rating'] = unseen_songs['songID'].apply(
    lambda song_id: model_with_bias.predict(uid=199988, iid=song_id).est
)

top_recommendations = unseen_songs.sort_values(by='predicted_rating', ascending=False)
top_recommendations.head(10)

<ipython-input-28-12c89c101a10>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unseen_songs.loc[:, 'predicted_rating'] = unseen_songs['songID'].apply(


,songID,predicted_rating
32,92881,4.575922
5,134732,4.551057
102,25182,4.445392
0,90409,4.416128
31,55240,4.395692
48,13859,4.381602
51,72309,4.362053
123,132189,4.296068
18,68572,4.296020
28,113954,4.285001


In [29]:
# Non-Biased Model
# Non-Biased Model
unseen_songs.loc[:, 'predicted_rating'] = unseen_songs['songID'].apply(
    lambda song_id: model_without_bias.predict(uid=199988, iid=song_id).est
)

top_recommendations = unseen_songs.sort_values(by='predicted_rating', ascending=False)
top_recommendations.head(10)

,songID,predicted_rating
167,122065,2.617296
9,86341,2.557825
123,132189,2.217376
0,90409,2.105204
25,22763,1.802834
31,55240,1.555804
76,2263,1.542970
71,36561,1.525503
14,55622,1.377999
8,79622,1.276928


----------------------------------------------

$$ Wish \space you \space all \space the \space best \space ♡ $$
$$ Abdelrahman \space Eid $$